In [17]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# !kaggle datasets download priyamchoksi/1-million-reddit-jokes-rjokes

In [3]:
# !unzip 1-million-reddit-jokes-rjokes.zip -d dataset

In [4]:
df = pd.read_csv('dataset/one-million-reddit-jokes.csv')

In [5]:
df = df[(df['selftext'] != '[removed]')]
df = df[(df['selftext'] != '[deleted]')]
df.drop(columns=['domain', 'url'], inplace=True)

# Deleting the rows with null selftext as the number of rows is very less
df.dropna(subset=['selftext'], inplace=True)
df = df[df['selftext'] != '']

# Deleting the rows with null title as the number of rows is very less
df.dropna(subset=['title'], inplace=True)
df = df[df['title'] != '']

# Removing the columns: type, id, subreddit.id, subreddit.name, subreddit.nsfw
df.drop(columns=['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw'], inplace=True)
df.head()

,created_utc,permalink,selftext,title,score
0,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15
7,1585784461,https://old.reddit.com/r/Jokes/comments/ftbeb4...,Oo..lala...,What did the French man say to the attractive ...,2
10,1585784081,https://old.reddit.com/r/Jokes/comments/ftbahg...,"Yo momma's so fat, that when she went to the z...",Yo Mama,0


In [6]:
df.isna().sum()

,0
created_utc,0
permalink,0
selftext,0
title,0
score,0


In [7]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
custom_stopwords = [
    'cant', 'dont', 'even', 'get', 'go', 'going', 'im', 'ive', 'like', 'really', 'want',
    'also', 'anyone', 'back', 'would', 'things', 'one', 'much', 'feeling', 'got', 'know',
    'never', 'since', 'always', 'something', 'started', 'still', 'take', 'anything', 'could',
    'getting', 'last', 'think', 'else', 'every', 'everything', 'lot', 'make', 'way', 'ago',
    'better', 'didnt', 'see', 'someone', 'went', 'well', 'thoughts', 'well', 'trying', 'first',
    'right', 'thought', 'person', 'ever', 'self', 'without', 'best', 'looking', 'day', 'ever',
    'broke', 'without', 'best', 'looking', 'ever', 'said', 'shes', 'hes', 'made', 'talk', 'tell',
    'thing', 'days', 'away', 'around', 'try', 'thats', 'times', 'saying', 'wasnt', 'doesnt', 'say',
    'wanted', 'nothing', 'anymore', 'keep', 'two', 'point', 'let', 'makes', 'asked', 'talking',
    'normal', 'actually', 'come', 'sure', 'many', 'done', 'find', 'though', 'end', 'left', 'deal',
    'maybe', 'stop', 'give', 'whats', 'etc', 'almost', 'little', 'everyone', 'tired', 'long', 'post',
    'id', 'able', 'another', 'says', "i", "am", "did", "does", "do", "was", "were", "will", "are", "be", "been",
    "being", "have", "has", "had", "having", "shall", "should", "would", "may", "might", "must", "can", "could",
    "ought", "dare", "need", "used", "seem", "see", "look",
    "appear", "sound", "smell", "taste", "feel", "become", "get", "grow", "turn", "prove", "remain","removed", "deleted",
    "ampx200b",
    "asks", "told", "oh", "put", "next"
]

def clean_text(text):
    # Lowercasing
    text = text.lower()
    # Remove Punctuation and Special Characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove Stop Words
    stop_words = set(stopwords.words('english')) | set(custom_stopwords)
    words = word_tokenize(text)
    text = ' '.join([word for word in words if word not in stop_words])
    return text

# Applying to selftext and title columns
df['title'] = df['title'].apply(clean_text)
df['selftext'] = df['selftext'].apply(clean_text)


In [9]:
df['X'] = df['title'] + ' ' + df['selftext']
df.head()

,created_utc,permalink,selftext,title,score,X
0,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,corona covered foreskin exposed viruses,soooo glad circumcised,2,soooo glad circumcised corona covered foreskin...
1,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,called google sheets,google platform recording bowel movements,9,google platform recording bowel movements call...
2,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,vacuum snore sex april fools wife,difference wife vacuum,15,difference wife vacuum vacuum snore sex april ...
7,1585784461,https://old.reddit.com/r/Jokes/comments/ftbeb4...,oolala,french man attractive teletubby,2,french man attractive teletubby oolala
10,1585784081,https://old.reddit.com/r/Jokes/comments/ftbahg...,yo mommas fat zoo hippos jealous,yo mama,0,yo mama yo mommas fat zoo hippos jealous


In [10]:
import torch
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

# Step 1: Check if GPU is available and load the model
# Check GPU availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Using device: {device}")

# Load SentenceTransformer model and move it to GPU
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)  # Pre-trained model

# Step 2: Generate embeddings for the jokes
jokes = df['X'].tolist()  # Extract jokes
sentence_embeddings = model.encode(jokes, batch_size=32, show_progress_bar=True, device=device)  # Generate embeddings on GPU

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/17942 [00:00<?, ?it/s]

In [11]:
import pickle

# Save the sentence embeddings to a pickle file
with open('embeddings/sentence_embeddings.pkl', 'wb') as f:
    pickle.dump(sentence_embeddings, f)

In [12]:
# with open('embeddings/sentence_embeddings.pkl', 'rb') as f:
#     loaded_embeddings = pickle.load(f)
# loaded_embeddings.shape

In [13]:
# # Step 3: Elbow Analysis to Determine Optimal Number of Clusters
# inertia = []
# k_values = range(1, 11)  # Test cluster counts from 1 to 10

# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(sentence_embeddings)  # Clustering is done on CPU
#     inertia.append(kmeans.inertia_)

# # Plot the Elbow Graph
# plt.figure(figsize=(8, 5))
# plt.plot(k_values, inertia, marker='o')
# plt.title('Elbow Method for Optimal Clusters')
# plt.xlabel('Number of Clusters')
# plt.ylabel('Inertia')
# plt.xticks(k_values)
# plt.grid()
# plt.show()

In [14]:
optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['cluster'] = kmeans.fit_predict(sentence_embeddings)  # Assign cluster labels

# Save the clustered data
df.to_csv('clustered_jokes_semantic.csv', index=False)

# View a sample of the clustered jokes
print(df[['X', 'cluster']].head())


                                                    X  cluster
0   soooo glad circumcised corona covered foreskin...        1
1   google platform recording bowel movements call...        1
2   difference wife vacuum vacuum snore sex april ...        0
7              french man attractive teletubby oolala        0
10           yo mama yo mommas fat zoo hippos jealous        2


In [19]:
new_df = df[['X','cluster']]
new_df = new_df.sample(frac=0.1,random_state=42)
new_df.head()

,X,cluster
693240,androids dream electric sheep unless theyre welsh,2
745616,escape country middle east distance travel iran,1
620542,useless piece skin womans vagina called woman,0
995074,good ole boy driving pickup truck south caroli...,3
439884,fencing joke reddit turned riposte edit ill en...,3


In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Check GPU availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Using device: {device}")

# Step 1: Prepare Data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    new_df['X'].tolist(),
    new_df['cluster'].tolist(),
    test_size=0.2,
    random_state=42
)

train_data = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_data = Dataset.from_dict({'text': test_texts, 'label': test_labels})

# Step 2: Load Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=new_df['cluster'].nunique()
).to(device)  # Move model to GPU

# Tokenize the text data
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_data = train_data.remove_columns(['text'])
test_data = test_data.remove_columns(['text'])

# Set format for PyTorch tensors
train_data.set_format('torch')
test_data.set_format('torch')

# Step 3: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./SavedModels",
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",        # Save the model at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,  # Ensure best model is loaded at the end
    report_to="none",              # Disable Weights & Biases (optional)
)


# Step 4: Define Trainer
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)

# Step 5: Train the Model
trainer.train()

# Step 6: Evaluate the Model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Step 7: Predict Cluster for a New Joke
def predict_cluster(joke):
    inputs = tokenizer(joke, truncation=True, padding='max_length', max_length=128, return_tensors="pt").to(device)
    outputs = model(**inputs)
    predicted_cluster = torch.argmax(outputs.logits, dim=1).item()
    return predicted_cluster

new_joke = "Why don't skeletons fight each other? They don't have the guts."
predicted_cluster = predict_cluster(new_joke)
print(f"Predicted cluster: {predicted_cluster}")


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/45929 [00:00<?, ? examples/s]

Map:   0%|          | 0/11483 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.458400,0.316600,0.880867,0.880728
2,0.264800,0.346833,0.886354,0.886376
3,0.064700,0.461216,0.886963,0.887005


Evaluation Results: {'eval_loss': 0.3166002035140991, 'eval_accuracy': 0.8808673691544021, 'eval_f1': 0.8807281041594298, 'eval_runtime': 76.0949, 'eval_samples_per_second': 150.904, 'eval_steps_per_second': 9.436, 'epoch': 3.0}
Predicted cluster: 1


In [21]:
# Save the model
model.save_pretrained('saved_model')

In [22]:

!zip -r saved_model.zip saved_model
!zip -r embeddings.zip embeddings
!zip -r clustered_jokes_semantic.csv.zip clustered_jokes_semantic.csv
from google.colab import files
files.download('saved_model.zip')
files.download('embeddings.zip')
files.download('clustered_jokes_semantic.csv.zip')


  adding: saved_model/ (stored 0%)
  adding: saved_model/model.safetensors (deflated 7%)
  adding: saved_model/config.json (deflated 53%)
  adding: embeddings/ (stored 0%)
  adding: embeddings/sentence_embeddings.pkl (deflated 7%)
  adding: embeddings/.ipynb_checkpoints/ (stored 0%)
  adding: clustered_jokes_semantic.csv (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>